Notebook for topological and template features calculation.
Based on [the code](https://github.com/danchern97/tda4atd/blob/main/features_calculation_by_thresholds.ipynb). 

# Topological features

In [1]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from multiprocessing import Pool
import os
from time import sleep
from tqdm.notebook import tqdm
import pandas as pd
from math import ceil
import numpy as np
import torch
import gzip
import networkx as nx
from pathlib import Path

In [2]:
def cutoff_matrix(matrix, ntokens):
    """Return normalized submatrix of first n_tokens"""
    matrix = matrix[:ntokens, :ntokens]
    matrix /= matrix.sum(axis=1, keepdims=True)
    return matrix

In [3]:
def get_filtered_mat_list(adj_matrix, thresholds_array, ntokens):
    """
    Converts adjacency matrix with real weights into list of binary matrices.
    For each threshold, those weights of adjacency matrix, which are less than
    threshold, get "filtered out" (set to 0), remained weights are set to ones.

    Args:
        adj_matrix (np.array[float, float])
        thresholds_array (iterable[float])
        ntokens (int)

    Returns:
        filtered_matricies (list[np.array[int, int]])
    """
    filtered_matrices = []
    for thr in thresholds_array:
        filtered_matrix = adj_matrix.copy()
        filtered_matrix = cutoff_matrix(filtered_matrix, ntokens)
        filtered_matrix[filtered_matrix < thr] = 0
        filtered_matrix[filtered_matrix >= thr] = 1
        filtered_matrices.append(filtered_matrix.astype(np.int8))
    return filtered_matrices


def adj_m_to_nx_list(adj_matrix, thresholds_array, ntokens, no_mat_output=False):
    """
    Converts adjacency matrix into list of unweighted digraphs, using filtering
    process from previous function.

    Args:
        adj_matrix (np.array[float, float])
        thresholds_array (iterable[float])
        ntokens (int)

    Returns:
        nx_graphs_list (list[nx.MultiDiGraph])
        filt_mat_list(list[np.array[int, int]])

    """
    #     adj_matrix = adj_matrix[:length,:length]
    filt_mat_list = get_filtered_mat_list(adj_matrix, thresholds_array, ntokens)
    nx_graphs_list = []
    for mat in filt_mat_list:
        nx_graphs_list.append(nx.from_numpy_matrix(np.array(mat),
                                                   create_using=nx.MultiDiGraph()))
    if no_mat_output:
        return nx_graphs_list, []
    else:
        return nx_graphs_list, filt_mat_list


def adj_ms_to_nx_lists(adj_matricies, \
                       thresholds_array, \
                       ntokens_array, \
                       verbose=False, \
                       no_mat_output=False):
    """
    Executes adj_m_to_nx_list for each matrix in adj_matricies array, arranges
    the results. If verbose==True, shows progress bar.

    Args:
        adj_matricies (np.array[float, float])
        thresholds_array (iterable[float])
        verbose (bool)

    Returns:
        nx_graphs_list (list[nx.MultiDiGraph])
        filt_mat_lists (list[list[np.array[int,int]]])
    """
    graph_lists = []
    filt_mat_lists = []

    iterable = range(len(adj_matricies))
    if verbose:
        iterable = tqdm(range(len(adj_matricies)),
                        desc="Calc graphs list")
    for i in iterable:
        g_list, filt_mat_list = adj_m_to_nx_list(adj_matricies[i], \
                                                 thresholds_array, \
                                                 ntokens_array[i], \
                                                 no_mat_output=no_mat_output)
        graph_lists.append(g_list)
        filt_mat_lists.append(filt_mat_lists)

    return graph_lists, filt_mat_lists


def count_stat(g_listt_j, function=nx.weakly_connected_components, cap=500):
    stat_amount = 0
    for _ in function(g_listt_j):
        stat_amount += 1
        if stat_amount >= cap:
            break
    return stat_amount


def count_weak_components(g_listt_j, cap=500):
    return count_stat(g_listt_j, function=nx.weakly_connected_components, cap=cap)


def count_strong_components(g_listt_j, cap=500):
    return count_stat(g_listt_j, function=nx.strongly_connected_components, cap=cap)


def count_simple_cycles(g_listt_j, cap=500):
    return count_stat(g_listt_j, function=nx.simple_cycles, cap=cap)


def count_b1(g_listt_j, cap=500):
    return count_stat(g_listt_j, function=nx.cycle_basis, cap=cap)


def dim_connected_components(graph_lists, strong=False, verbose=False, cap=500):
    """
    Calculates number of connected components for each graph in list
    of lists of digraphs. If strong==True, calculates strongly connected
    components, otherwise calculates weakly connected components.
    If verbose==True, shows progress bar.

    Args:
        graph_lists (list[list[nx.MultiDiGraph]])
        strong (bool)
        verbose (bool)

    Returns:
        w_lists (list[list[int])
    """
    w_lists = []  # len == len(w_graph_lists)
    iterable = range(len(graph_lists))
    if verbose:
        iterable = tqdm(range(len(graph_lists)),
                        desc="Calc weak comp")
    for i in iterable:
        g_list = graph_lists[i]
        w_cmp = []
        for j in range(len(g_list)):
            if strong:
                w_cmp.append(count_strong_components(g_list[j], cap=cap))
            else:
                w_cmp.append(count_weak_components(g_list[j], cap=cap))
        w_lists.append(w_cmp)
    return w_lists


def dim_simple_cycles(graph_lists, verbose, cap=500):
    """
    Calculates number of simple cycles for each graph in list
    of lists of digraphs. If verbose==True, shows progress bar.

    Args:
        graph_lists (list[list[nx.MultiDiGraph]])
        verbose (bool)

    Returns:
        c_lists (list[list[int])
    """
    c_lists = []  # len == len(pos_w_graph_lists)
    iterable = range(len(graph_lists))
    if verbose:
        iterable = tqdm(range(len(graph_lists)),
                        desc="Calc cycles")
    for i in iterable:
        g_list = graph_lists[i]
        c = []
        for j in range(len(g_list)):
            c.append(count_simple_cycles(g_list[j], cap=cap))
        c_lists.append(c)
    if verbose:
        logger = logging.getLogger()
        flat = [x for l in c_lists for x in l]
        logger.debug('dim_simple_cycles: min=%f mean=%f max=%f,  ratio of cap = %d / %d' %
                     (
                         np.min(c_lists), np.mean(c_lists), np.max(c_lists), len([x for x in flat if x == cap]),
                         len(flat)))

    return c_lists


def dim_b1(graph_lists, verbose, cap=500):
    b1_lists = []  # len == len(pos_w_graph_lists)
    iterable = range(len(graph_lists))
    if verbose:
        iterable = tqdm(range(len(graph_lists)),
                        desc="Calc b1 (undirected graphs)")
    for i in iterable:
        g_list = graph_lists[i]
        b1 = []
        for j in range(len(g_list)):
            b1.append(count_b1(nx.Graph(g_list[j].to_undirected()), cap=cap))
        b1_lists.append(b1)
    return b1_lists


def b0_b1(graph_lists, verbose):
    b0_lists = []
    b1_lists = []  # len == len(pos_w_graph_lists)
    iterable = range(len(graph_lists))
    if verbose:
        iterable = tqdm(range(len(graph_lists)),
                        desc="Calc b0, b1")
    for i in iterable:
        g_list = graph_lists[i]
        b0 = []
        b1 = []
        for j in range(len(g_list)):
            g = nx.Graph(g_list[j].to_undirected())
            w = nx.number_connected_components(g)
            e = g.number_of_edges()
            v = g.number_of_nodes()
            b0.append(w)
            b1.append(e - v + w)
        b0_lists.append(b0)
        b1_lists.append(b1)
    return b0_lists, b1_lists


def edges_f(graph_lists, verbose):
    """
    Calculates number of edges for each graph in list
    of lists of digraphs. If verbose==True, shows progress bar.

    Args:
        graph_lists (list[list[nx.MultiDiGraph]])
        verbose (bool)

    Returns:
        e_lists (list[list[int])
    """
    e_lists = []  # len == len(pos_w_graph_lists)
    iterable = range(len(graph_lists))
    if verbose > 2:
        iterable = tqdm(range(len(graph_lists)),
                        desc="Calc edges number")
    for i in iterable:
        g_list = graph_lists[i]
        e = []
        for j in range(len(g_list)):
            e.append(g_list[j].number_of_edges())
        e_lists.append(e)
    return e_lists


def v_degree_f(graph_lists, verbose):
    """
    Calculates number of edges for each graph in list
    of lists of digraphs. If verbose==True, shows progress bar.

    Args:
        graph_lists (list[list[nx.MultiDiGraph]])
        verbose (bool)

    Returns:
        v_lists (list[list[int])
    """
    v_lists = []  # len == len(pos_w_graph_lists)
    iterable = range(len(graph_lists))
    if verbose > 2:
        iterable = tqdm(range(len(graph_lists)),
                        desc="Calc average vertex degree")
    for i in iterable:
        g_list = graph_lists[i]
        v = []
        for j in range(len(g_list)):
            degrees = g_list[j].degree()
            degree_values = [v for k, v in degrees]
            sum_of_edges = sum(degree_values) / float(len(degree_values))
            v.append(sum_of_edges)
        v_lists.append(v)
    return v_lists


def chordality_f(graph_lists, verbose):
    """
    Checks whether the graph is chordal or not for each graph in list of lists of graphs.
    If verbose==True, shows progress bar.

    Args:
        graph_lists (list[list[nx.MultiDiGraph]])
        verbose (bool)

    Returns:
        ch_lists (list[list[int])
    """
    ch_lists = []  # len == len(pos_w_graph_lists)
    iterable = range(len(graph_lists))
    for i in iterable:
        g_list = graph_lists[i]
        ch = []
        for j in range(len(g_list)):
            g = nx.Graph(g_list[j].to_undirected())
            # print(g)
            # print(g.edges())
            g.remove_edges_from(nx.selfloop_edges(g))
            ch_i = nx.is_chordal(g)
            ch.append(int(ch_i))
        ch_lists.append(ch)
    return ch_lists
def max_matching_f(graph_lists, verbose):
    """
    Calculates max matching size for each graph in list
    of lists of graphs. If verbose==True, shows progress bar.

    Args:
        graph_lists (list[list[nx.MultiDiGraph]])
        verbose (bool)

    Returns:
        max_m_lists (list[list[int])
    """
    max_m_lists = []  # len == len(pos_w_graph_lists)
    iterable = range(len(graph_lists))
    if verbose > 2:
        iterable = tqdm(range(len(graph_lists)),
                        desc="Calc max matching of the graph")
    for i in iterable:
        g_list = graph_lists[i]
        max_m = []
        for j in range(len(g_list)):
            g = nx.Graph(g_list[j].to_undirected())
            m_i = nx.maximal_matching(g)
            max_m.append(len(m_i))
        max_m_lists.append(max_m)
    return max_m_lists

def count_top_stats(adj_matricies,
                    thresholds_array,
                    ntokens_array,
                    stats_to_count={"s", "e", "c", "v", "b0b1"},
                    stats_cap=500,
                    verbose=False):
    """
    The main function for calculating topological invariants. Unites the
    functional of all functions above.
    Args:
        adj_matricies (np.array[float, float, float, float, float])
        thresholds_array (list[float])
        stats_to_count (str)
        stats_cap (int)
        verbose (bool)
    Returns:
        stats_tuple_lists_array (np.array[float, float, float, float, float])
    """
    stats_tuple_lists_array = []

    for layer_of_interest in tqdm(range(adj_matricies.shape[1])):
        stats_tuple_lists_array.append([])
        for head_of_interest in range(adj_matricies.shape[2]):
            adj_ms = adj_matricies[:, layer_of_interest, head_of_interest, :, :]
            g_lists, _ = adj_ms_to_nx_lists(adj_ms,
                                            thresholds_array=thresholds_array,
                                            ntokens_array=ntokens_array,
                                            verbose=verbose)
            feat_lists = []
            if "s" in stats_to_count:
                feat_lists.append(dim_connected_components(g_lists,
                                                           strong=True,
                                                           verbose=verbose,
                                                           cap=stats_cap))
            if "w" in stats_to_count:
                feat_lists.append(dim_connected_components(g_lists,
                                                           strong=False,
                                                           verbose=verbose,
                                                           cap=stats_cap))
            if "e" in stats_to_count:
                feat_lists.append(edges_f(g_lists, verbose=verbose))
            if "v" in stats_to_count:
                feat_lists.append(v_degree_f(g_lists, verbose=verbose))
            if "c" in stats_to_count:
                feat_lists.append(dim_simple_cycles(g_lists,
                                                    verbose=verbose,
                                                    cap=50))

            if "b0b1" in stats_to_count:
                b0_lists, b1_lists = b0_b1(g_lists, verbose=verbose)
                feat_lists.append(b0_lists)
                feat_lists.append(b1_lists)
            if "m" in stats_to_count:
                feat_lists.append(max_matching_f(g_lists,
                                                    verbose=verbose))
            if "k" in stats_to_count:
                feat_lists.append(chordality_f(g_lists,
                                                    verbose=verbose))
            stats_tuple_lists_array[-1].append(tuple(feat_lists))

    stats_tuple_lists_array = np.asarray(stats_tuple_lists_array,
                                         dtype=np.float16)
    return stats_tuple_lists_array


def function_for_v(list_of_v_degrees_of_graph):
    return sum(map(lambda x: np.sqrt(x * x), list_of_v_degrees_of_graph))

In [4]:
subset = "test_sub"

In [5]:
model_path = "./bert-base-cased-en-cola_32_3e-05_lr_0.01_decay_balanced/"

In [6]:
input_dir = output_dir = './'
layers_of_interest = list(range(24))
data = pd.read_csv("./data/en-cola/" + subset + '.csv')

In [7]:
def get_token_length(batch_texts):
    inputs = tokenizer.batch_encode_plus(batch_texts,
       return_tensors='pt',
       add_special_tokens=True,
       max_length=MAX_LEN,             # Max length to truncate/pad
       pad_to_max_length=True,         # Pad sentence to max length
       truncation=True
    )
    inputs = inputs['input_ids'].numpy()
    n_tokens = []
    indexes = np.argwhere(inputs == tokenizer.pad_token_id)
    for i in range(inputs.shape[0]):
        ids = indexes[(indexes == i)[:, 0]]
        if not len(ids):
            n_tokens.append(MAX_LEN)
        else:
            n_tokens.append(ids[0, 1])
    return n_tokens

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
max_tokens_amount  = 64
MAX_LEN = max_tokens_amount

In [9]:
data['tokenizer_length'] = get_token_length(list(data['sentence'].values))
ntokens_array = data['tokenizer_length'].values

/opt/tljh/user/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
stats_name = "s_e_v_c_b0b1_m_k"
thresholds_array = [0.025, 0.05, 0.1, 0.25, 0.5, 0.75]
thrs = len(thresholds_array)
layers_of_interest = [i for i in range(12)]
stats_file = model_path + 'features/' + subset + "" + stats_name + "_array_" + str(thrs) + '.npy'
stats_file             

'./bert-base-cased-en-cola_32_3e-05_lr_0.01_decay_balanced/features/test_subs_e_v_c_b0b1_m_k_array_6.npy'

In [11]:
def order_files(path, subset):
    files_path = Path(path)
    files = list(filter(lambda y: (y.is_file() and subset in str(y)), files_path.iterdir()))
    files = [str(_) for _ in files]
    files = sorted(files, key=lambda x: int(x.split('_')[-1].split('of')[0][4:].strip()))
    return files

In [12]:
output_dir=model_path
attn_dir = model_path + "/attentions/"
adj_filenames = order_files(path=attn_dir, subset=subset)
adj_filenames

['bert-base-cased-en-cola_32_3e-05_lr_0.01_decay_balanced/attentions/test_sub_part1of1.npy.gz']

In [13]:
def split_matricies_and_lengths(adj_matricies, ntokens_array, num_of_workers):
    splitted_adj_matricies = np.array_split(adj_matricies, num_of_workers)
    splitted_ntokens = np.array_split(ntokens_array, num_of_workers)
    assert all([len(m)==len(n) for m, n in zip(splitted_adj_matricies, splitted_ntokens)]), "Split is not valid!"
    return zip(splitted_adj_matricies, splitted_ntokens)

In [14]:
num_of_workers = os.cpu_count()
pool = Pool(num_of_workers)
num_of_workers

40

In [15]:
batch_size = 10 # batch size
number_of_batches = ceil(len(data['sentence']) / batch_size)
DUMP_SIZE = 100 # number of batches to be dumped
batched_sentences = np.array_split(data['sentence'].values, number_of_batches)
number_of_files = ceil(number_of_batches / DUMP_SIZE)

In [16]:
debug=False

In [17]:
stats_name = "s_e_v_c_b0b1_m_k" # "c"
stats_cap = 500
stats_tuple_lists_array = []
for i, filename in enumerate(tqdm(adj_filenames, 
                                  desc='Calculating topological features')):
    with gzip.GzipFile(filename, 'rb') as f:
        adj_matricies = np.load(f, allow_pickle=True)
        ntokens = ntokens_array[i*batch_size*DUMP_SIZE : (i+1)*batch_size*DUMP_SIZE]
    if debug:
        stats_tuple_lists_array_part = count_top_stats(adj_matricies, thresholds_array, ntokens, stats_name.split("_"), stats_cap, verbose=2) 
    else:
        splitted = split_matricies_and_lengths(adj_matricies, ntokens, num_of_workers)
        args = [(m, thresholds_array, ntokens, stats_name.split("_"), stats_cap) for m, ntokens in splitted]
        stats_tuple_lists_array_part = pool.starmap(
            count_top_stats, args
        )
    stats_tuple_lists_array.append(np.concatenate([_ for _ in stats_tuple_lists_array_part], axis=3))

Calculating topological features:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
stats_tuple_lists_array = np.concatenate(stats_tuple_lists_array, axis=3)

In [19]:
!mkdir features

mkdir: cannot create directory ‘features’: File exists


In [20]:
stats_file

'./bert-base-cased-en-cola_32_3e-05_lr_0.01_decay_balanced/features/test_subs_e_v_c_b0b1_m_k_array_6.npy'

In [21]:
np.save(stats_file, np.concatenate(stats_tuple_lists_array, axis=3))

# Template features

In [22]:
model_dir_ = "./bert-base-cased-en-cola_32_3e-05_lr_0.01_decay_balanced/"
for data_subset in ["test_sub"]:
    d_dir = f"./data/en-cola/{data_subset}.csv"
    !python src.template_features.py --model_dir $model_dir_ --data_file $d_dir --num_of_workers 40

Using custom data configuration default-99873f36c864e5e9
Found cached dataset csv (/home/jupyter-test/.cache/huggingface/datasets/csv/default-99873f36c864e5e9/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Template Feature Calc: 100%|██████████████████████| 1/1 [00:16<00:00, 16.82s/it]
